In [94]:
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

^C


In [2]:
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [95]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [96]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [5]:
!cp '/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/Jobs_NYC_Postings.7z' .
!cp '/content/drive/MyDrive/Colab Notebooks/store/hust/big data integration/work/large_naukri_data.7z' .

In [6]:
!apt install p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [97]:
!p7zip -d  large_naukri_data.7z 1> /dev/null
!p7zip -d  Jobs_NYC_Postings.7z 1> /dev/null

/usr/bin/p7zip: cannot read large_naukri_data.7z
/usr/bin/p7zip: cannot read Jobs_NYC_Postings.7z


In [98]:
indian_jobs = spark.read.json('large_naukri_data.json')
old = indian_jobs.schema.names
new = [item.replace(' ', '_') for item in old]
indian_jobs = indian_jobs.toDF(*new)
indian_jobs.createOrReplaceTempView('indian_view')

In [99]:
nyc_jobs = spark.read.csv('Jobs_NYC_Postings.csv', header=True)
old = nyc_jobs.schema.names
new = [item.replace(' ', '_') for item in old]
new = [item.replace('#', '_') for item in new]
new = [item.replace('/', '_') for item in new]
new = [item.replace('-', '_') for item in new]
nyc_jobs = nyc_jobs.toDF(*new)
nyc_jobs.createOrReplaceTempView('nyc_view')
#

In [100]:
indian_jobs.schema.names

['_corrupt_record',
 '_id',
 'company_name',
 'crawled_at',
 'currency',
 'description',
 'industry',
 'job_id',
 'job_type',
 'locations',
 'max_experience',
 'max_salary',
 'min_experience',
 'min_salary',
 'pg',
 'posted_at',
 'ppg',
 'role',
 'salary_label',
 'skills',
 'title',
 'ug',
 'url',
 'vacany',
 'view_count',
 'walkin']

In [80]:
nyc_jobs.schema.names

['Job_ID',
 'Agency',
 'Posting_Type',
 '__Of_Positions',
 'Business_Title',
 'Civil_Service_Title',
 'Title_Classification',
 'Title_Code_No',
 'Level',
 'Job_Category',
 'Full_Time_Part_Time_indicator',
 'Career_Level',
 'Salary_Range_From',
 'Salary_Range_To',
 'Salary_Frequency',
 'Work_Location',
 'Division_Work_Unit',
 'Job_Description',
 'Minimum_Qual_Requirements',
 'Preferred_Skills',
 'Additional_Information',
 'To_Apply',
 'Hours_Shift',
 'Work_Location_1',
 'Recruitment_Contact',
 'Residency_Requirement',
 'Posting_Date',
 'Post_Until',
 'Posting_Updated',
 'Process_Date']

In [107]:
from pyspark.sql.functions import when
id_col = ['industry', 'job_type', 'min_experience','max_experience',
          'min_salary', 'max_salary', 'skills', 'vacany', 'role']
id_df = spark.sql(f'select {",".join(id_col)} from indian_view where industry is not null')
id_df = id_df.withColumn('max_salary', id_df['max_salary'] * 0.012)
id_df = id_df.withColumn('min_salary', id_df['min_salary'] * 0.012)
id_df = id_df.withColumn('job_type', when(id_df.job_type == 'fulltime', 'F').otherwise('P'))
id_df.show(5)
#

+--------------------+--------+--------------+--------------+----------+----------+--------------------+------+--------------------+
|            industry|job_type|min_experience|max_experience|min_salary|max_salary|              skills|vacany|                role|
+--------------------+--------+--------------+--------------+----------+----------+--------------------+------+--------------------+
|IT-Software, Soft...|       F|             4|             6|    6000.0|   10800.0|Procurement | Mar...|     1|Purchase/Vendor D...|
|Construction, Eng...|       F|             8|            12|     600.0|     600.0|Information Secur...|     0|               Other|
|KPO, Research, An...|       F|             1|             3|    1800.0|    2400.0|Clinical | Health...|     5|     Medical Officer|
| Internet, Ecommerce|       F|             2|             6|    9600.0|   15600.0|Product managemen...|     1|Head/VP/GM-Techno...|
|BPO, Call Centre,...|       F|             0|             5|    3600

In [111]:
ny_col = [ 'Job_Category as industry', 'Full_Time_Part_Time_indicator as job_type',
          'Salary_Range_From as min_salary', 'Salary_Range_To as max_salary',
           '__Of_Positions as vacany', 'Business_Title as role' ]
ny_df = spark.sql(f'select {",".join(ny_col)} from nyc_view')
ny_df.show(5)

+--------------------+--------+----------+----------+------+--------------------+
|            industry|job_type|min_salary|max_salary|vacany|                role|
+--------------------+--------+----------+----------+------+--------------------+
|Health Policy, Re...|       F|    105746|    150000|     1|Executive Directo...|
|Administration & ...|       F|     57490|     85912|     4|ASSOCIATE INVESTI...|
|Constituent Servi...|       F|     60889|     78923|     1|Eyeglass Distribu...|
|Engineering, Arch...|       F|     58682|    159671|     1|BOB- Administrati...|
|Engineering, Arch...|       F|     58682|    134570|     1|     Deputy Director|
+--------------------+--------+----------+----------+------+--------------------+
only showing top 5 rows



In [17]:
indian_avg_max_salary = id_df.agg({'max_salary': 'avg'}).collect()[0][0]
indian_avg_min_salary = id_df.agg({'min_salary': 'avg'}).collect()[0][0]

print(indian_avg_max_salary, indian_avg_min_salary)
#

nyc_avg_max_salary = ny_df.agg({'max_salary': 'avg'}).collect()[0][0]
nyc_avg_min_salary = ny_df.agg({'min_salary': 'avg'}).collect()[0][0]

print(nyc_avg_max_salary, nyc_avg_min_salary)
#

13113.652993399532 7899.255805314812
91036.06056827023 65905.8791151818


In [70]:
id_description = id_df.select('description').distinct().collect()
#
id_d = id_description[0].description

https://blog.finxter.com/4-best-ways-to-strip-html-tags-from-a-python-string/

In [69]:
import re

def remove_html_tags(text):
    html_pattern = re.compile('<.*?>')
    clean_text = re.sub(html_pattern, '', text)
    return clean_text

print(remove_html_tags(id_d))

 Sales Officer (General Trade)1. Candidate must be from FMCG Experience2. Candidate must have experience handling Primary & Secondary sales3. Must have keen market knowledge & products4. To coordinate with the Manager for daily sales & targets5. To visit assigned market on daily basis6. Maintaining good business relation with the Distributors & Outlets7. Minimum 2-4 years of experience in FMCG - General Trade


In [78]:
id_df.select('industry').distinct().collect()[2].industry, id_df.select('skills').distinct().collect()[2].skills


('Government, Defence',
 'Java | Life Cycle | Defect Management | Testng | Test Scenarios | Cucumber | Test Cases')

In [92]:
ny_df.select('industry').distinct().collect()[16].industry, ny_df.select('skills').distinct().collect()[16].skills


('Constituent Services & Community Programs Technology, Data & Innovation Policy, Research & Analysis Social Services',
 'Experience in environmental sciences, water testing, supervising field staff, and designing drinking and/or wastewater plans.  Ability to climb ladders and work in confined spaces, work in high altitudes (building roofs) in order to perform water testing.   Pass medical clearance and FIT testing for wearing proper PPE respirators.   Complete and pass Agency required trainings and be in compliance with position related certificates and licenses.  Strong interpersonal and customer service skills, with the ability to interact at all levels in addition to exceptional written and communication skills.  Demonstrate excellent organizational skills and attention to detail.  Proficiency in the Microsoft Office suite (Word, Excel, Access, PowerPoint)')

In [104]:
nyc_description = ny_df.select('Business_Title').distinct().collect()
#
tmp = nyc_description#[0].Business_Title
#
tmp

[Row(Business_Title='Project Executive'),
 Row(Business_Title='SPECIAL ASSISTANT TO THE CHIEF OF HOMELESS PREVENT...'),
 Row(Business_Title='Engineering Auditor'),
 Row(Business_Title='Records Access & Chief Privacy Officer'),
 Row(Business_Title='College Aide Intern 2023'),
 Row(Business_Title='Certified IT Developer (Application)'),
 Row(Business_Title='MECHANICAL ENGINEERING INTERN'),
 Row(Business_Title='Associate Water Use Inspector Level I'),
 Row(Business_Title='Director of Data Science and Innovation'),
 Row(Business_Title='Administrative Assistant for the Division of Asset Management'),
 Row(Business_Title='BOB-Project Manager/Engineer-In-Charge'),
 Row(Business_Title='Research and Policy Analyst'),
 Row(Business_Title='Executive Director, Strategic Initiatives'),
 Row(Business_Title='AREA ENGINEER â\x80\x93 ELECTRICAL ENGINEERING'),
 Row(Business_Title='Associate Commissioner, Security Operations'),
 Row(Business_Title='Early Intervention Official Designee, Bureau of Early In

In [105]:
ny_df.select('industry').distinct().collect()

[Row(industry='Administration & Human Resources Legal Affairs Policy, Research & Analysis Public Safety, Inspections, & Enforcement'),
 Row(industry='Engineering, Architecture, & Planning Building Operations & Maintenance Public Safety, Inspections, & Enforcement'),
 Row(industry='Constituent Services & Community Programs Green Jobs Building Operations & Maintenance'),
 Row(industry='Legal Affairs Policy, Research & Analysis Public Safety, Inspections, & Enforcement'),
 Row(industry='Constituent Services & Community Programs Finance, Accounting, & Procurement Social Services'),
 Row(industry='Constituent Services & Community Programs'),
 Row(industry='Building Operations & Maintenance'),
 Row(industry='Engineering, Architecture, & Planning Policy, Research & Analysis Public Safety, Inspections, & Enforcement'),
 Row(industry='Constituent Services & Community Programs Finance, Accounting, & Procurement'),
 Row(industry='Building Operations & Maintenance Policy, Research & Analysis'),
 R

https://stackoverflow.com/questions/13110629/decoding-utf-8-strings-in-python

In [42]:
tmp.encode('utf-8').decode('cp1252')

'APPLICANTS MUST BE PERMANENT IN THE ADMINISTRATIVE STAFF ANALYST CIVIL SERVICE TITLE OR BE PERMANENT IN A COMPARABLE TITLE ELIGIBLE FOR 6.1.9 TITLE CHANGE.   The Department of Social Services (DSS) is comprised of the administrative units of the New York City Human Resources Administration (HRA) and the Department of Homeless Services (DHS). Human Resources Administration is dedicated to fighting poverty and income inequality by providing New Yorkers in need with essential benefits such as Food Assistance and Emergency Rental Assistance. Department of Homeless Services is committed to preventing and addressing homelessness in New York City by employing a variety of innovative strategies to help families and individuals successfully exit shelter and return to self- sufficiency as quickly as possible.  The Office of Fiscal Operations is DSS/HRA/DHSÃ¢Â€Â™s primary payments and accounting office, and the core responsibilities are carried out through the operations of the Bureau of Account

In [56]:
from pyspark.sql.functions import decode
from pyspark.sql.functions import encode
nyc_jobs_update_1 = nyc_jobs.withColumn('Job_Description', decode(encode(nyc_jobs['Job_Description'], 'utf-8'), 'cp1252'))
#

In [66]:
nyc_jobs_update_1.createOrReplaceTempView('nyc_view')
#ny_df = spark.sql(f'select Job_Description from nyc_view')
#nyc_description = ny_df.select('Job_Description').distinct().collect()
#
#tmp = nyc_description[1].Job_Description
#tmp
#

"APPLICANTS MUST BE PERMANENT IN THE ASSOCIATE BENEFITS OPPORTUNITY SPECIALIST TITLE OR PERMANENT IN THE BENEFITS OPPORTUNITY SPECIALIST TITLE FOR AT LEAST ONE YEAR; THIS IS A PROVISIONAL APPOINTMENT, WHEN A TEST BECOMES AVAILABLE IN THE ASSOCIATE BENEFITS OPPORTUNITY SPECIALIST (ABOS) TITLE, YOU MUST TAKE AND PASS THE EXAM TO REMAIN IN THE ABOS TITLE.  Family Independence Administration's provides temporary help to individuals and families with social service and economic needs to assist them in reaching self-sufficiency via essential and diverse programs/services, such as: temporary cash assistance, SNAP, childcare, eviction prevention services, adult protective services, job training, employment plans/support, domestic violence assistance, child support enforcement, etc.  Under the direction of the Associate Benefits Opportunity Specialist II, the Associate Benefits Opportunity Specialist I (ABOS I) is responsible for supervising a team of BOS staff, who provide various functions of